In [1]:
import numpy as np
import pandas as pd
import math
import random
import csv

import pickle

from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import pairwise_kernels

from params import *
from functions import *

In [2]:
f = open('results/results_online_SHR_CLUSTER.txt', 'w+')

for SEED in range(0,10):

    set_seed(SEED)
    data_incoming = pd.read_pickle(f'data/data_incoming_{SEED}.pickle')
    lambda_h, scaler, feature_cluster_list, error_cluster_list, size_cluster_list = pickle.load(open(f'results/results_offline_SHR_CLUSTER_{SEED}.pickle','rb'))

    objective_function_true_total = 0
    remaining_budget = B
    
    uncover_set = set()
    
    for n in range(0,N_incoming):

        fare = data_incoming.loc[n, 'fare']
        compensation = data_incoming.loc[n, 'compensation']
        
        feature = data_incoming.loc[n, feature_name].values
        kernel = pairwise_kernels(feature_cluster_list, scaler.transform(np.array(feature).reshape(1, -1)), metric='rbf', gamma=gamma).flatten()
        error = np.sum(kernel*size_cluster_list*error_cluster_list)/np.sum(kernel*size_cluster_list)      
        
        l_best = -np.inf
        i_best = 0
        j_best = 0

        for i in range(0, len(passenger_incentive_list)):
            for j in range(0, len(driver_incentive_list)):
                
                pred_prob = data_incoming.loc[n, f'pred_prob_{i}_{j}'] + error
                
                if (fare*passenger_incentive_list[i]+compensation*driver_incentive_list[j] > remaining_budget):
                    uncover_set.add(n)
                
                if (fare*passenger_incentive_list[i]+compensation*driver_incentive_list[j] <= remaining_budget and eta*fare*(1-passenger_incentive_list[i]) < compensation*(1+driver_incentive_list[j])):
                    l_current = pred_prob-lambda_h*pred_prob*(fare*passenger_incentive_list[i]+compensation*driver_incentive_list[j])
                    if (l_current > l_best):
                        l_best = l_current
                        i_best = i
                        j_best = j
            
        objective_function_true_total += data_incoming.loc[n, f'true_prob_{i_best}_{j_best}']
        remaining_budget -= data_incoming.loc[n, f'true_prob_{i_best}_{j_best}'] * (fare*passenger_incentive_list[i_best]+compensation*driver_incentive_list[j_best])

    f.write(f'{SEED}, SHR-CLUSTER, {objective_function_true_total}, {1-len(uncover_set)/N_incoming}\n')
    
f.close()